In [1]:
import pandas as pd
from datetime import datetime, timedelta
import geopy.distance
import ipyleaflet as L
from ipywidgets import Layout
df = pd.read_hdf('data/AEMPData/aempdata_subset.h5')
df = df.drop_duplicates()

In [2]:
#Problem, determining unique trucks

In [3]:
df

,Make,Model,Latitude,Longitude,Hour,FuelUnits,FuelConsumed,FuelUnitsLast24,FuelConsumedLast24,OdometerUnits,Odometer,datetime,ID
0,HITACHI,ZX210LC-6,59.9743,10.3312,PT4843H56M24S,None,None,liter,25,kilometer,0,2022-02-26 00:00:00+00:00,0.0
1,HITACHI,ZX210LC-6,59.9401,10.4184,PT3731H37M12S,None,None,liter,24,kilometer,0,2022-02-26 00:00:00+00:00,1.0
2,HITACHI,ZX210LC-6,59.9519,10.3749,PT3135H23M24S,None,None,liter,47,kilometer,0,2022-02-26 00:00:00+00:00,2.0
3,HITACHI,ZX210LC-6,59.9471,10.3841,PT7918H13M48S,None,None,liter,149,kilometer,0,2022-02-26 00:00:00+00:00,3.0
4,HITACHI,ZX300-7,59.9761,10.3216,PT2617H36M,None,None,liter,202,kilometer,0,2022-02-26 00:00:00+00:00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,Volvo,A45G,59.9644,10.3217,P52DT7H27M,liter,14443,None,None,kilometer,31,2022-06-01 04:50:01+00:00,32.0
1991,Volvo,L110H,59.9528,10.3736,P237DT22H42M,liter,43540,None,None,kilometer,25949,2022-06-01 21:12:46+00:00,29.0
1992,Volvo,L110H,59.9750,10.3311,P54DT11H18M,liter,9480,None,None,kilometer,4741,2022-06-01 21:46:51+00:00,26.0
1993,Volvo,L110H,59.9636,10.3244,P37DT7H36M,liter,5809,None,None,kilometer,2239,2022-06-01 20:50:45+00:00,30.0


In [4]:
df.groupby(['Make','Model']).size().reset_index().rename(columns={0:'count'})

#Dumpere
    #CAT	745-04 ID: 24
    #Volvo	A45G ID: 32
#Trucks

df_dumper = df[(df['Make']=='CAT') & (df['Model']=='745-04') | (df['Make']=='Volvo') & (df['Model']=='A45G')]
#df_volvo_dumper = df[(df['Make']=='Volvo') & (df['Model']=='A45G')]



In [5]:
df_dumper

,Make,Model,Latitude,Longitude,Hour,FuelUnits,FuelConsumed,FuelUnitsLast24,FuelConsumedLast24,OdometerUnits,Odometer,datetime,ID
1013,CAT,745-04,59.96458,10.32222,P302DT12H,gallon,7842,None,None,miles,32132,2022-05-03 18:24:38+00:00,24.0
1024,CAT,745-04,59.96462,10.32237,P302DT13H,gallon,7845,None,None,miles,32132,2022-05-04 17:27:51+00:00,24.0
1035,CAT,745-04,59.96462,10.32237,P302DT13H,gallon,7845,None,None,miles,32132,2022-05-05 17:31:51+00:00,24.0
1046,CAT,745-04,59.96454,10.32237,P302DT18H,gallon,7870,None,None,miles,32146,2022-05-06 20:31:14+00:00,24.0
1057,CAT,745-04,59.96451,10.32237,P302DT18H,gallon,7875,None,None,miles,32149,2022-05-07 17:38:52+00:00,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,Volvo,A45G,59.96440,10.32190,P51DT3H12M,liter,14197,None,None,kilometer,20,2022-05-27 18:06:52+00:00,32.0
1967,Volvo,A45G,59.96440,10.32190,P51DT7H3M,liter,14270,None,None,kilometer,21,2022-05-28 11:53:58+00:00,32.0
1981,Volvo,A45G,59.96440,10.32190,P51DT17H36M,liter,14342,None,None,kilometer,24,2022-05-30 17:55:36+00:00,32.0
1985,Volvo,A45G,59.96440,10.32170,P52DT4H21M,liter,14443,None,None,kilometer,31,2022-05-31 16:19:33+00:00,32.0


In [6]:
date = datetime.strptime('2022-04-20 00:00:01+00:00', '%Y-%m-%d %H:%M:%S%z') #Choose one second after midnight to not include loads of vehicles with datetime at midnight
date_string_short = '04-20-2022' #MM-DD-ÅÅÅÅ
filtered_df = df_dumper[(df_dumper['datetime'] >= date) & (df_dumper['datetime'] < date+timedelta(days=1))]

In [7]:
filtered_df

,Make,Model,Latitude,Longitude,Hour,FuelUnits,FuelConsumed,FuelUnitsLast24,FuelConsumedLast24,OdometerUnits,Odometer,datetime,ID
1713,Volvo,A45G,59.9644,10.3217,P46DT10H42M,liter,12867,None,None,kilometer,43,2022-04-20 04:11:37+00:00,32.0


In [8]:
observed_dumper_lat_lon = (filtered_df.iloc[0,2],filtered_df.iloc[0,3])
observed_dumper_lat_lon

(59.9644, 10.3217)

In [9]:
registered_parking_spots = {}
for index, row in filtered_df.iterrows():
    registered_parking_spots[row.ID]=(row.Latitude, row.Longitude)

In [10]:
import dataloader
trips = dataloader.TripsLoader(date_string_short)
m = L.Map(layout=Layout(width='60%', height='700px'), center=[59.95, 10.3])
import ipyleaflet as L
for machine_id in trips._machines.keys():
        temp_machine = trips._machines[machine_id]
        if temp_machine.machine_type=='Dumper':
            for trip in temp_machine.trips:
                poly = L.Polyline(locations=trip.latlons)
                m.add_layer(poly)
aemp_point = L.Marker(location=(observed_dumper_lat_lon))
m.add_layer(aemp_point)

display(m)
m.save('2022-04-20_dumper.html', title='AEMP and trip data for dumpers')

                

Map(center=[59.95, 10.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [11]:
#Logikk: Henter alle posisjoner på biler fra AEMP data 00:00:01 og 24 timer frem i tid
#        Henter alle siste registrerte posisjoner på biler dagen
#        Hvis turen ender morgenen 8. mars tror jeg sjåfør har glemt å avslutte tur og siste registrerte posisjon er kanskje dårlig
import ipyleaflet as L
from ipywidgets import Layout
m = L.Map(layout=Layout(width='60%', height='700px'), center=[59.95, 10.3])
for key in possible_pos:
    # Create a custom icon with the specified color
    icon = L.AwesomeIcon(name='check', marker_color='red', icon_color='black', spin=False)
    load_mark = L.Marker(location=(possible_pos[key].lat,possible_pos[key].lon), title=f'Trip: '+str(key), icon=icon)
    m.add_layer(load_mark)
for key in registered_parking_spots.keys():
    print("En gang")
    load_mark = L.Marker(location=registered_parking_spots[key], title=f'AEMP: ' +str(key))
    m.add_layer(load_mark)
display(m)

NameError: name 'possible_pos' is not defined

In [ ]:
registered_parking_spots.keys()

dict_keys([24.0])